In [ ]:
# folium package

In [2]:
"""
run_query(\"""SELECT
        f_user_id as id, followed_user_id as fid,
        c_lat as lat, c_lon as lon, f_lat as flat, f_lon as flon, dist
        INTO alex..follower_graph
        FROM jszule..follower_2dir_coord\""")
"""

'\nrun_query("""SELECT\n        f_user_id as id, followed_user_id as fid,\n        c_lat as lat, c_lon as lon, f_lat as flat, f_lon as flon, dist\n        INTO alex..follower_graph\n        FROM jszule..follower_2dir_coord""")\n'

In [3]:
#connection = pymssql.connect("future1", timeout=300)

#try:
 #   with connection.cursor() as cursor:
      #  sql = """CREATE TABLE alex..Twitter_User (
       #             ID bigint NOT NULL,
        #            lon real NOT NULL,
         #           lat real NOT NULL
          #          PRIMARY KEY (ID));"""
   #     sql2 = """INSERT INTO alex..Twitter_User (ID, lon, lat)
    #            SELECT DISTINCT id as ID, lon, lat
     #       FROM alex..follower_graph;"""
  #      cursor.execute(sql2)

    # connection is not autocommit by default. So you must commit to save changes.
    # connection.commit()

#finally:
 #   connection.close()

In [4]:
import pymssql
import numpy as np
import pandas as pd

def run_query(query):
    connection = pymssql.connect("future1", timeout=300) # DB server
    cursor = connection.cursor()
    cursor.execute(query)
    return pd.DataFrame(cursor.fetchall())

followers = run_query("""SELECT follower.fid, follower.flon,
follower.flat, follower.dist, twitter_user.lon, twitter_user.lat,
twitter_user.ID
FROM alex..Twitter_User as twitter_user
INNER JOIN alex..follower_graph as follower
ON twitter_user.ID = follower.id
ORDER BY twitter_user.ID
""")
followers.columns = ['fid', 'flon', 'flan', 'dist', 'lon', 'lat', 'id']
followers

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



OperationalError: (20009, b'DB-Lib error message 20009, severity 9:\nUnable to connect: Adaptive Server is unavailable or does not exist (future1:1433)\nNet-Lib error during Connection timed out (110)\n')

In [ ]:
followers.values.shape, followers.columns

In [ ]:
def build_dictionary(df):
    values = df[['fid', 'flon', 'flan', 'lon', 'lat', 'id']].values
    followers = dict()
    for value in values:
        followers[value[5]] = []
    for value in values:
        followers[value[5]].append(value[0:5])
    return followers

In [ ]:
follower_dict = build_dictionary(followers)
len(follower_dict)

In [ ]:
followed_users = follower_dict.keys()
len(followed_users)

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
one_user = list(followed_users)[0]

In [ ]:
model = DBSCAN()

In [ ]:
followers_matrix = np.array(follower_dict[12])[:, 1:3]
print(followers_matrix.shape)

In [ ]:
import matplotlib.pyplot as plt
from astropy.coordinates import spherical_to_cartesian, cartesian_to_spherical

def preict_from_most_dense_cluster(data):
    clf = DBSCAN(eps=5./6371., min_samples=5, 
                     algorithm='ball_tree', metric='haversine').fit(np.radians(data.values))

    distinct_labels = set(clf.labels_)
    if -1 in distinct_labels:
        distinct_labels.remove(-1)
    
    labeledData = [0]*len(distinct_labels)
    for label in distinct_labels:
        labeledData[label] = []
    for ind, label in enumerate(clf.labels_):
        if label >= 0:
            labeledData[label].append(data.values[ind, :])
    labeledData = np.array(labeledData)
    
    sizes = []
    for ind in range(labeledData.shape[0]):
        sizes.append(len(labeledData[ind]))
        
    def transform2cart(arr):
        lon, lat = arr
        x, y, z = spherical_to_cartesian(6371, np.radians(lat), np.radians(lon))
        return [x, y, z]
    
    def transform2geo(arr):
        x, y, z = arr
        r, lat, lon = cartesian_to_spherical(x, y, z)
        lat = np.degrees(lat.value) if np.degrees(lat.value) < 180 else (-180 + np.degrees(lat.value) - 180)
        lon = np.degrees(lon.value) if np.degrees(lon.value) < 180 else (-180 + np.degrees(lon.value) - 180)
        return np.array([lon, lat])
    
    transformedData = []
    
    if(len(sizes) == 0):
        for row in np.array(data.values):
            transformedData.append(transform2cart(row))
        return transform2geo(np.mean(transformedData, axis=0)) # naive method
    
    max_ind = np.argmax(sizes)

    for row in np.array(labeledData[max_ind]):
        transformedData.append(transform2cart(row))
    return transform2geo(np.mean(transformedData, axis=0))


In [ ]:
preict_from_most_dense_cluster(pd.DataFrame(followers_matrix)) # actual -122.405975	37.783192

In [ ]:
np.mean(followers_matrix, axis=0) # naive

In [ ]:
def transform2cart(arr):
    lon, lat = arr
    x, y, z = spherical_to_cartesian(6371, np.radians(lat), np.radians(lon))
    return np.array([x, y, z])

true_and_predicted = dict()
for ind, key in enumerate(list(follower_dict.keys())[0:50000]):
    followers_matrix = np.array(follower_dict[key])
    prediction = preict_from_most_dense_cluster(pd.DataFrame(followers_matrix[:, 1:3]))
    dist = np.linalg.norm(
        transform2cart(np.array(prediction)) - transform2cart(np.array(followers_matrix[0, 3:5])))
    true_and_predicted[key] = np.array([prediction[0], prediction[1], followers_matrix[0, 3:5][0], 
                                        followers_matrix[0, 3:5][1], dist])
    print(".", end='')
    if((ind+1) % 5000 == 0): 
        print()
        print('%d already processed!' % ind)

In [ ]:
data = []
for key in true_and_predicted.keys():
    data.append(np.array(true_and_predicted[key]).flatten())
data = pd.DataFrame(np.array(data))
data.values.shape, len(true_and_predicted)

In [ ]:
data.head(n=25)

In [ ]:
data[[0, 1]].to_csv(path_or_buf='pred.txt', sep='\t')

In [ ]:
data[[2, 3]].to_csv(path_or_buf='actu.txt', sep='\t')

In [ ]:
data.to_csv(path_or_buf='both.csv')

In [ ]:
closer_than_five = data[4] < 15
filtered_data = data[closer_than_five]
filtered_data.shape

In [ ]:
filtered_data.head(n=15)

In [ ]:
filtered_data.to_csv(path_or_buf='both.csv')